# Smeared Data Reduction

This notebook compiles reduced SANS and USANS data collected at the NIST Center for Neutron Research over two trips that occured in August and December of 2019. The USANS data is smeared, and therefore, is kept separate from the SANS data so that when fitting the data, separate smearing conditions can be applied to the appropriate datasets.

Samples are indentified with a sample key, numbered 1-35, 40-43 for the August batch and 101-114, 201-206, 301-310, 401-410 and 501-506 for the December batch. The SANS data can be found in this supplemental information data reduction and analysis folder at the location `./sans/Aug2019_SANS` and `./sans/Dec2019_SANS`. Also included are csv files with additional information matching the sample keys to the sample names, as well as identifying the file numbers to the appropriate samples (in the August batch). Please see the main README file with additional information about all sections of this supplemental information data reduction and analysis folder.

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

from bumps.names import *
from bumps.fitters import fit

import sasmodels

from sasmodels.core import load_model
from sasmodels.bumps_model import Model, Experiment
from sasmodels.data import load_data, plot_data, empty_data1D
from sasmodels.direct_model import DirectModel

import sas

We will filter out the following sample keys as the samples were not pressed for a sufficient amount of time.

In [2]:
filters = [201,202,204,410,308]

Loading August 2019 SANS data.

In [3]:
# data directory
aug_dir = './Aug2019_SANS/'

# 'SamplefileNumbers.csv' connects the samples to the appropriate files
file_nos = pd.read_csv(aug_dir + 'SampleFileNumbers.csv')
ng0_files = {}   # first configuration on VSANS instrument
ng5_files = {}   # second configuration on VSANS instrument

for index,row in file_nos.iterrows():
    sam_no = int(row['Sample'])
    ng0_files[sam_no] = 'sans' + str(row['NG0']) + '.ABS'
    ng5_files[sam_no] = 'sans' + str(row['NG5']) + '.ABS'
    
# loading the sans data for each sample using the AnalySAS package
sans_data = {}
for key in ng0_files.keys():
    ng0_data = np.loadtxt(aug_dir + ng0_files[key], skiprows=12)[:,0:4]
    ng5_data = np.loadtxt(aug_dir + ng5_files[key], skiprows=12)[:,0:4]
        
    # remove significantly smeared data points at the cross-over between configurations
    mask = np.where(ng5_data[:,2]/ng5_data[:,1] < 0.75)
    ng5_data = ng5_data[mask, :][0]
    
    data = np.vstack((ng0_data, ng5_data))
    
    # filter out negative intensity (noise)
    mask = np.where(data[:,1] > 0)
    data = data[mask,:][0]
    
    # removes signficantly smeared data pionts at the cross-over between configurations
    if key not in filters:
        sans_data[key] = data


Loading December 2019 SANS data.

In [4]:
dec_dir = './Dec2019_SANS/'
sample_info = pd.read_csv(dec_dir + 'SampleFileNumbers.csv')

files = [file for file in os.listdir(dec_dir) if file[-4:] == '.ABS']
keys = [int(file[:3]) for file in files]

for key, file in zip(keys, files):
    data = np.loadtxt(dec_dir + file, skiprows=6)[:,0:4]
    
    # remove significantly smeared data points at the cross-over between configurations
    mask = np.where(data[:,2]/data[:,1] < 0.75)
    data = data[mask, :][0]
    
    # filter out negative intensity (noise)
    mask = np.where(data[:,1] > 0)
    data = data[mask,:][0]
    
    if key not in filters:
        sans_data[key] = data

Saving the smeared SANS data in txt files for future analysis.

In [5]:
save_loc = './Smeared_Data_20200629'
os.makedirs(save_loc, exist_ok=True)

for key, data in sans_data.items():
    
    np.savetxt(save_loc + '/CMW' + str(key) + '_SANS.txt', data, header = '<X> <Y> <dY> <dX>')


Loading the smeared USANS data from both August and December batches.

In [6]:
usans_data = {}
files = [x for x in os.listdir('./USANS_smeared') if 'CMW' in x]

for file in files:
    key = int(file.split('_')[0][3:])
    data = np.loadtxt('USANS_smeared/' + file, skiprows=7)
    data = data[:,0:3]
    
    # trimming the data to q-range collected and removing noise where intensity is negative
    mask = np.where((data[:,0] > 0.00005) & (data[:,1] > 0))
    data = data[mask,:][0]
    
    # removing noisy data where signal was weak above background
    mask = np.where((data[:,2]*100/data[:,1]) < 75)
    
    # sample 35 (a polystyrene background) required additional trimming due to low signal
    if (key == 35 or key == 501):
        mask = np.where(((data[:,2]*100/data[:,1]) < 75) & (data[:,0]<1e-3))
        
    data = data[mask,:][0]

    # removing filtered samples and 310 (not enough signal above background)
    if key not in filters and key != 310:
        usans_data[key] = data

Saving the smeared SANS data in txt files for future analysis.

In [7]:
for key, data in usans_data.items():
    if key not in filters:
        np.savetxt(save_loc + '/CMW' + str(key) + '_USANS.txt', data, header = '<X> <Y> <dY>')